In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt


All the patients suffered heart attacks at some point in the past. Some are still alive and some are not. The survival and still-alive variables, when taken together, indicate whether a patient survived for at least one year following the heart attack.
The problem addressed by past researchers was to predict from the other variables whether or not the patient will survive at least one year. The most difficult part of this problem is correctly predicting that the patient will NOT survive. (Part of the difficulty seems to be the size of the data set.)
survival -- the number of months patient survived (has survived, if patient is still alive). Because all the patients had their heart attacks at different times, it is possible that some patients have survived less than one year but they are still alive. Check the second variable to confirm this. Such patients cannot be used for the prediction task mentioned above.


   1. survival -- the number of months patient survived (has survived, if patient is still alive).  Because all the patients had their heart attacks at different times, it is possible that some patients have survived less than one year but they are still alive.  Check the second variable to confirm this.  Such patients cannot be used for the prediction task mentioned above.
   2. still-alive -- a binary variable.  0=dead at end of survival period, 1 means still alive
   3. age-at-heart-attack -- age in years when heart attack occurred
   4. pericardial-effusion -- binary. Pericardial effusion is fluid around the heart.  0=no fluid, 1=fluid
   5. fractional-shortening -- a measure of contracility around the heart lower numbers are increasingly abnormal
   6. epss -- E-point septal separation, another measure of contractility.  Larger numbers are increasingly abnormal.
   7. lvdd -- left ventricular end-diastolic dimension.  This is a measure of the size of the heart at end-diastole. Large hearts tend to be sick hearts.
   8. wall-motion-score -- a measure of how the segments of the left ventricle are moving
   9. wall-motion-index -- equals wall-motion-score divided by number of segments seen.  Usually 12-13 segments are seen in an echocardiogram.  Use this variable INSTEAD of the wall motion score.
   10. mult -- a derivate var which can be ignored
   11. name -- the name of the patient (I have replaced them with "name")
   12. group -- meaningless, ignore it
   13. alive-at-1 -- Boolean-valued. Derived from the first two attributes. 0 means patient was either dead after 1 year or had been followed for less than 1 year.  1 means patient was alive at 1 year.



In [ ]:
#Load data from csv and store it in a dataframe
data = pd.read_csv("echocardiogram.data", skiprows=1, delimiter=',')
display(data)

,19,0,72,0.1,0.380,6,4.100,14,1.700,0.588,name,1,0.2
0,16,0,55,0,0.260,4,3.420,14,1,1,name,1,0
1,57,0,60,0,0.253,12.062,4.603,16,1.450,0.788,name,1,0
2,19,1,57,0,0.160,22,5.750,18,2.250,0.571,name,1,0
3,26,0,68,0,0.260,5,4.310,12,1,0.857,name,1,0
4,13,0,62,0,0.230,31,5.430,22.5,1.875,0.857,name,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,7.5,1,64,0,0.24,12.9,4.72,12,1,0.857,name,?,?
126,41,0,64,0,0.28,5.40,5.47,11,1.10,0.714,name,?,?
127,36,0,69,0,0.20,7.00,5.05,14.5,1.21,0.857,name,?,?
128,22,0,57,0,0.14,16.1,4.36,15,1.36,0.786,name,?,?


In [ ]:
features=['Survival', 'Still-Alive', 'Age-At-Heart-Attack', 'Pericardial-effusion','Fractional-Shortening', 'EPSS', 'LVDD', 'Wall-Motion-Score', 'Wall-Motion-Index', 'Mult', 'Name', 'Group', 'Alive-At']
data = pd.read_csv("echocardiogram.data", header=None, names=features, delimiter=',')
data.head()

,Survival,Still-Alive,Age-At-Heart-Attack,Pericardial-effusion,Fractional-Shortening,EPSS,LVDD,Wall-Motion-Score,Wall-Motion-Index,Mult,Name,Group,Alive-At
0,11,0,71,0,0.260,9,4.600,14,1,1,name,1,0
1,19,0,72,0,0.380,6,4.100,14,1.700,0.588,name,1,0
2,16,0,55,0,0.260,4,3.420,14,1,1,name,1,0
3,57,0,60,0,0.253,12.062,4.603,16,1.450,0.788,name,1,0
4,19,1,57,0,0.160,22,5.750,18,2.250,0.571,name,1,0


In [ ]:
#Remove useless data
columns_to_remove = ['Mult', 'Name', 'Group']
newdata = data.drop(columns=columns_to_remove, axis=0)
newdata.head()

,Survival,Still-Alive,Age-At-Heart-Attack,Pericardial-effusion,Fractional-Shortening,EPSS,LVDD,Wall-Motion-Score,Wall-Motion-Index,Alive-At
0,11,0,71,0,0.260,9,4.600,14,1,0
1,19,0,72,0,0.380,6,4.100,14,1.700,0
2,16,0,55,0,0.260,4,3.420,14,1,0
3,57,0,60,0,0.253,12.062,4.603,16,1.450,0
4,19,1,57,0,0.160,22,5.750,18,2.250,0


## a) Descriptive analysis of the database.

In [ ]:
# Number of entries in the dataset
print("This data frame is made of", newdata.shape[0],"entries, and", newdata.shape[1], "features" )
data=newdata

This data frame is made of 132 entries, and 10 features


In [ ]:
#Types of data
print(data.dtypes)

Survival                 object
Still-Alive              object
Age-At-Heart-Attack      object
Pericardial-effusion     object
Fractional-Shortening    object
EPSS                     object
LVDD                     object
Wall-Motion-Score        object
Wall-Motion-Index        object
Alive-At                 object
dtype: object


In [ ]:
(data=='?').sum(axis=0)

Survival                  2
Still-Alive               1
Age-At-Heart-Attack       5
Pericardial-effusion      1
Fractional-Shortening     8
EPSS                     15
LVDD                     11
Wall-Motion-Score         4
Wall-Motion-Index         1
Alive-At                 58
dtype: int64

In [ ]:
(data=='0').sum(axis=0)

Survival                   0
Still-Alive               88
Age-At-Heart-Attack        0
Pericardial-effusion     107
Fractional-Shortening      0
EPSS                       7
LVDD                       0
Wall-Motion-Score          0
Wall-Motion-Index          0
Alive-At                  50
dtype: int64

Remove all the '?' of the dataframe

In [ ]:
data.replace('?', pd.NA, inplace=True)
#data.dropna(axis=0, how='any', inplace=True)
display(data)

,Survival,Still-Alive,Age-At-Heart-Attack,Pericardial-effusion,Fractional-Shortening,EPSS,LVDD,Wall-Motion-Score,Wall-Motion-Index,Alive-At
0,11,0,71,0,0.260,9,4.600,14,1,0
1,19,0,72,0,0.380,6,4.100,14,1.700,0
2,16,0,55,0,0.260,4,3.420,14,1,0
3,57,0,60,0,0.253,12.062,4.603,16,1.450,0
4,19,1,57,0,0.160,22,5.750,18,2.250,0
...,...,...,...,...,...,...,...,...,...,...
127,7.5,1,64,0,0.24,12.9,4.72,12,1,NaN
128,41,0,64,0,0.28,5.40,5.47,11,1.10,NaN
129,36,0,69,0,0.20,7.00,5.05,14.5,1.21,NaN
130,22,0,57,0,0.14,16.1,4.36,15,1.36,NaN


Replace 0 of the EPSS column with the mean value of the dataframe

In [ ]:
data=data.apply(lambda col:pd.to_numeric(col, errors='coerce')).astype('Float64')
print(data.dtypes)
print(data.mean(axis=0))

Survival                 Float64
Still-Alive              Float64
Age-At-Heart-Attack      Float64
Pericardial-effusion     Float64
Fractional-Shortening    Float64
EPSS                     Float64
LVDD                     Float64
Wall-Motion-Score        Float64
Wall-Motion-Index        Float64
Alive-At                 Float64
dtype: object
Survival                 22.182923
Still-Alive               0.328244
Age-At-Heart-Attack      62.925425
Pericardial-effusion      0.183206
Fractional-Shortening     0.216734
EPSS                     12.164769
LVDD                      4.763157
Wall-Motion-Score        14.438125
Wall-Motion-Index         1.382748
Alive-At                  0.324324
dtype: float64


In [ ]:
data.fillna(data.mean(axis=0), inplace=True)
print(data.iloc[28])
(data==data.mean(axis=0)).sum(axis=0)

Survival                      0.25
Still-Alive                    1.0
Age-At-Heart-Attack           75.0
Pericardial-effusion           0.0
Fractional-Shortening     0.216734
EPSS                     12.164769
LVDD                      4.763157
Wall-Motion-Score        14.438125
Wall-Motion-Index              1.0
Alive-At                  0.324324
Name: 28, dtype: Float64


<ipython-input-118-5de1737ffe7a>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.fillna(data.mean(axis=0), inplace=True)


Survival                  2
Still-Alive               1
Age-At-Heart-Attack       5
Pericardial-effusion      1
Fractional-Shortening     8
EPSS                     15
LVDD                     11
Wall-Motion-Score         4
Wall-Motion-Index         1
Alive-At                  0
dtype: int64